# CRS Data Loader v2

Simple 6-phase workflow for loading CRS master data:
1. **Tenant & Branding** - Organization setup and UI customization
2. **Boundaries** - Administrative hierarchy (State → District → Block)
3. **Common Masters** - Departments, designations, complaint types
4. **Employees** - Staff accounts with roles and assignments
5. **Localizations** (Optional) - Bulk load translations for new languages
6. **Workflow** - Complaint state machine (APPLY → ASSIGN → RESOLVE)

---

In [ ]:
import warnings
import pandas as pd
import requests
from openpyxl import load_workbook
from crs_loader import CRSLoader

In [ ]:
# Configuration - Edit these values
URL = "https://chakshu-digit.egov.theflywheel.in"
USERNAME = "ADMIN"
PASSWORD = "eGov@123"
TENANT_ID = "statea"  # Root tenant for login
TARGET_TENANT = "statea"  # Target tenant for data (can be child like "statea.citya")

loader = CRSLoader(URL)
loader.login(username=USERNAME, password=PASSWORD, tenant_id=TENANT_ID)

In [ ]:
# Phase 1 - Tenant & Branding
loader.load_tenant("templates/Tenant And Branding Master.xlsx", target_tenant=TARGET_TENANT)

In [ ]:
# Phase 2 - Boundaries
loader.load_boundaries("templates/Boundary_Master.xlsx", target_tenant=TARGET_TENANT, hierarchy_type="REVENUE")

In [ ]:
# Phase 3 - Common Masters
loader.load_common_masters("templates/Common and Complaint Master.xlsx", target_tenant=TARGET_TENANT)

In [ ]:
# Phase 4 - Employees
loader.load_employees("templates/Employee_Master_Dynamic_statea.xlsx", target_tenant=TARGET_TENANT)

---

## Phase 5: Localizations (Optional)

Load bulk localization messages for a new language. Use this when you need to add Hindi, Tamil, Punjabi, or other regional languages.

**Prerequisites:**
- You need a localization Excel file with a `Localization` sheet
- Required columns: `Code`, `Message`, `Locale` (e.g., `hi_IN`, `pa_IN`)
- Optional columns: `Module`

**To add a new language:**
1. Get the localization template from `templates/localization.xlsx`
2. Fill in translations for the new locale
3. Run the cell below with `language_label` and `locale_code` to enable the language in the UI dropdown

In [ ]:
# Phase 5 - Localizations (Optional)
# For bulk translations - upload localization Excel with translated messages

# Option A: Just upload messages (no new language in dropdown)
loader.load_localizations("templates/localization.xlsx", target_tenant=TARGET_TENANT)

# Option B: Upload messages AND enable new language in UI dropdown
# Uncomment and edit the values below:

# loader.load_localizations(
#     "templates/localization.xlsx",
#     target_tenant=TARGET_TENANT,
#     language_label="ਪੰਜਾਬੀ",   # Display name in UI (e.g., "Hindi", "ਪੰਜਾਬੀ", "తెలుగు")
#     locale_code="pa_IN"         # Locale code (e.g., "hi_IN", "pa_IN", "te_IN")
# )

---

## Phase 6: Workflow Configuration

Configure the PGR complaint workflow state machine from a JSON file. This defines the complaint lifecycle:
- **APPLY** → Citizen/CSR creates complaint
- **PENDINGFORASSIGNMENT** → GRO assigns to field worker  
- **PENDINGATLME** → Field worker resolves or reassigns
- **RESOLVED/REJECTED** → Terminal states with rating option

**Template:** Use `../default-data-handler/src/main/resources/PgrWorkflowConfig.json` as a starting point.

The JSON file should have a `BusinessServices` array. Use `{tenantid}` as a placeholder - it will be replaced with your target tenant.

In [ ]:
# Phase 6 - Workflow
# Loads the PGR complaint workflow state machine from JSON file

# Default PGR workflow template (copy and modify as needed)
WORKFLOW_JSON = "../default-data-handler/src/main/resources/PgrWorkflowConfig.json"

loader.load_workflow(WORKFLOW_JSON, target_tenant=TARGET_TENANT)

---

## Rollback / Delete Data

Use these cells to undo data loading. Run only what you need to rollback.

In [ ]:
# Rollback Phase 2 - Delete Boundaries
# Deletes all boundary entities and relationships for the tenant
loader.delete_boundaries(TARGET_TENANT)

In [ ]:
# Rollback Phase 3 - Delete Common Masters
# Deletes departments, designations, and complaint types
loader.rollback_common_masters(TARGET_TENANT)

In [ ]:
# Rollback Phase 1 - Delete Tenant Config
# Deletes tenant and branding configuration
loader.rollback_tenant(TARGET_TENANT)

In [ ]:
# Delete specific MDMS schema data (granular control)
loader.delete_mdms("common-masters.Department", TARGET_TENANT)
loader.delete_mdms("common-masters.Designation", TARGET_TENANT)
loader.delete_mdms("RAINMAKER-PGR.ServiceDefs", TARGET_TENANT)

In [ ]:
# FULL RESET - Delete everything (MDMS + Boundaries)
# WARNING: This deletes ALL data for the tenant!
loader.full_reset("REVENUE", TARGET_TENANT)

**Note:** Employees (Phase 4) cannot be deleted via API - they are managed in HRMS and must be deactivated manually if needed.